# IDS Demonstration

## Importing required libraries

In [1]:
!git clone https://github.com/jirifilip/pyIDS.git


Cloning into 'pyIDS'...
remote: Enumerating objects: 1023, done.
remote: Counting objects: 100% (520/520), done.
remote: Compressing objects: 100% (357/357), done.
remote: Total 1023 (delta 259), reused 389 (delta 150), pack-reused 503
Receiving objects: 100% (1023/1023), 3.21 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (533/533), done.


In [2]:
%cd pyIDS


/content/pyIDS


In [3]:
pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [4]:
pip install pyarc --no-deps


In [5]:
pip install pyfim --no-deps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfim: filename=pyfim-6.28-cp310-cp310-linux_x86_64.whl size=644238 sha256=eafe89c934446156276cdaebd44c260d7c90c72a9df55af98dde14a6f9c43cec
  Stored in directory: /root/.cache/pip/wheels/96/0a/b3/c877bfa85c4cfe1baf3de4a89e1949382be09de5eabe49314f
Successfully built pyfim


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.algorithms.ids_classifier import mine_CARs
from pyarc.qcba.data_structures import QuantitativeDataFrame
from pyids.model_selection.coordinate_ascent import CoordinateAscent


In [8]:
from sklearn.model_selection import train_test_split
data = pd.read_csv("/content/transformed_cell_samples1.csv")
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)


In [9]:
quant_dataframe_train = QuantitativeDataFrame(data_train)
quant_dataframe_test = QuantitativeDataFrame(data_test)

In [10]:
cars = mine_CARs(data_train, 50, sample=False)

In [11]:
from pyids.rule_mining import RuleMiner
rm = RuleMiner()
rules = rm.mine_rules(data_train)

[CAR {Mitoses=-inf_to_1.5} => {Class=benign} sup: 0.83 conf: 0.00 len: 2, id: 4653, CAR {Mitoses=-inf_to_1.5} => {Class=malignant} sup: 0.83 conf: 0.00 len: 2, id: 4654, CAR {Normal_Nucleoli=-inf_to_2.5} => {Class=benign} sup: 0.69 conf: 0.00 len: 2, id: 4649, CAR {Normal_Nucleoli=-inf_to_2.5} => {Class=malignant} sup: 0.69 conf: 0.00 len: 2, id: 4650, CAR {Mitoses=-inf_to_1.5,Normal_Nucleoli=-inf_to_2.5} => {Class=benign} sup: 0.66 conf: 0.00 len: 3, id: 4651, CAR {Mitoses=-inf_to_1.5,Normal_Nucleoli=-inf_to_2.5} => {Class=malignant} sup: 0.66 conf: 0.00 len: 3, id: 4652, CAR {Single_Epi_Cell_Size=-inf_to_2.5} => {Class=benign} sup: 0.62 conf: 0.00 len: 2, id: 4641, CAR {Single_Epi_Cell_Size=-inf_to_2.5} => {Class=malignant} sup: 0.62 conf: 0.00 len: 2, id: 4642, CAR {Mitoses=-inf_to_1.5,Single_Epi_Cell_Size=-inf_to_2.5} => {Class=benign} sup: 0.60 conf: 0.00 len: 3, id: 4647, CAR {Mitoses=-inf_to_1.5,Single_Epi_Cell_Size=-inf_to_2.5} => {Class=malignant} sup: 0.60 conf: 0.00 len: 3, 

In [12]:
ids = IDS()
ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train, random_seed=5)

/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


In [13]:
ids.score_interpretability_metrics(quant_dataframe_test)

{'fraction_overlap': 0.1375,
 'fraction_classes': 1.0,
 'fraction_uncovered': 0.23571428571428577,
 'average_rule_width': 2.875,
 'ruleset_length': 16}

In [14]:
ids.score_auc(quant_dataframe_test)

/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


0.9836257309941521

## Parameter tuning

In [16]:
from pyids.model_selection import CoordinateAscent
from pyids.algorithms.ids import IDS
from pyids.algorithms import mine_CARs, mine_IDS_ruleset

from pyarc.qcba.data_structures import QuantitativeDataFrame

import pandas as pd
import numpy as np

df_iris = pd.read_csv("/content/transformed_cell_samples1.csv")
quant_df = QuantitativeDataFrame(df_iris)
cars = mine_CARs(df_iris, 20)


interpretability_bounds = dict(
    fraction_overlap=0.1,
    fraction_classes=1,
    fraction_uncovered=0.15,
    average_rule_width=8,
    ruleset_length=10
)

def is_solution_interpretable(metrics):
    print(metrics)
    return (
        metrics["fraction_overlap"] <= interpretability_bounds["fraction_overlap"] and
        metrics["fraction_classes"] >= interpretability_bounds["fraction_classes"] and
        metrics["fraction_uncovered"] <= interpretability_bounds["fraction_uncovered"] and
        metrics["average_rule_width"] <= interpretability_bounds["average_rule_width"] and
        metrics["ruleset_length"] <= interpretability_bounds["ruleset_length"]
    )

def solution_interpretability_distance(metrics):
    distance_vector = np.array([
        max(metrics["fraction_overlap"] - interpretability_bounds["fraction_overlap"], 0),
        max(interpretability_bounds["fraction_classes"] - metrics["fraction_classes"], 0),
        max(metrics["fraction_uncovered"] - interpretability_bounds["fraction_uncovered"], 0),
        max(metrics["average_rule_width"] - interpretability_bounds["average_rule_width"], 0),
        max(metrics["ruleset_length"] - interpretability_bounds["ruleset_length"], 0)
    ])
    return np.sum(distance_vector)
    #return np.linalg.norm(distance_vector)

def fmax(lambda_dict):
    print(lambda_dict)
    ids = IDS(algorithm="SLS")
    ids.fit(class_association_rules=cars, quant_dataframe=quant_df, lambda_array=list(lambda_dict.values()))

    metrics = ids.score_interpretability_metrics(quant_df)

    """
    if not is_solution_interpretable(metrics):
        distance = -solution_interpretability_distance(metrics)
        print(distance)
        return -distance
    """

    if not is_solution_interpretable(metrics):
        return 0

    auc = ids.score_auc(quant_df)

    print(auc)

    return auc



coord_asc = CoordinateAscent(
    func=fmax,
    func_args_ranges=dict(
        l1=(1, 400),
        l2=(1, 400),
        l3=(1, 400),
        l4=(1, 400),
        l5=(1, 400),
        l6=(1, 400),
        l7=(1, 400)
    ),
    ternary_search_precision=50,
    max_iterations=2
)

best_pars = coord_asc.fit()

df = pd.DataFrame(coord_asc.procedure_data)

{'l1': 34, 'l2': 155, 'l3': 321, 'l4': 194, 'l5': 266, 'l6': 48, 'l7': 309}


/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


{'fraction_overlap': 0.37635397506642143, 'fraction_classes': 1.0, 'fraction_uncovered': 0.41487839771101576, 'average_rule_width': 2.875, 'ruleset_length': 8}
using precision 50
{'l1': 134.0, 'l2': 155, 'l3': 321, 'l4': 194, 'l5': 266, 'l6': 48, 'l7': 309}


/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


{'fraction_overlap': 0.39856938483547927, 'fraction_classes': 1.0, 'fraction_uncovered': 0.4306151645207439, 'average_rule_width': 3.0, 'ruleset_length': 5}
{'l1': 267.0, 'l2': 155, 'l3': 321, 'l4': 194, 'l5': 266, 'l6': 48, 'l7': 309}


...

/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


{'fraction_overlap': 0.38681109067375163, 'fraction_classes': 1.0, 'fraction_uncovered': 0.3948497854077253, 'average_rule_width': 2.7142857142857144, 'ruleset_length': 7}
{'l1': 12.676268861454044, 'l2': 12.676268861454044, 'l3': 12.676268861454044, 'l4': 12.676268861454044, 'l5': 12.676268861454044, 'l6': 12.676268861454044, 'l7': 24.35253772290809}


/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


{'fraction_overlap': 0.3624225083452551, 'fraction_classes': 1.0, 'fraction_uncovered': 0.44921316165951364, 'average_rule_width': 2.6666666666666665, 'ruleset_length': 3}
{'l1': 12.676268861454044, 'l2': 12.676268861454044, 'l3': 12.676268861454044, 'l4': 12.676268861454044, 'l5': 12.676268861454044, 'l6': 12.676268861454044, 'l7': 12.676268861454044}
{'fraction_overlap': 0.37649022412970906, 'fraction_classes': 1.0, 'fraction_uncovered': 0.4263233190271817, 'average_rule_width': 2.75, 'ruleset_length': 4}


/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


# Best parameters obtained

In [17]:
pars = [round(x, 2) for x in best_pars]

pars_df = pd.DataFrame({
    "parametr": [ f"$\lambda_{i}$" for i in range(1, 8) ],
    "hodnota parametru": pars
})

print(pars_df.to_latex(index=False, escape=False))

\begin{tabular}{lr}
\toprule
   parametr &  hodnota parametru \\
\midrule
$\lambda_1$ &                 34 \\
$\lambda_2$ &                155 \\
$\lambda_3$ &                321 \\
$\lambda_4$ &                194 \\
$\lambda_5$ &                266 \\
$\lambda_6$ &                 48 \\
$\lambda_7$ &                309 \\
\bottomrule
\end{tabular}



<ipython-input-17-de770b53e833>:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pars_df.to_latex(index=False, escape=False))


In [18]:
ids = IDS()
ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train, lambda_array=best_pars)
ids.score_auc(quant_dataframe_test)

/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():


0.9578947368421052

In [19]:
ids.score_interpretability_metrics(quant_dataframe_test)

{'fraction_overlap': 0.37423469387755104,
 'fraction_classes': 1.0,
 'fraction_uncovered': 0.37857142857142856,
 'average_rule_width': 3.0,
 'ruleset_length': 8}

In [20]:
metric_df = pd.DataFrame([
    {"Data": "Training", "Accuracy": ids.score(quant_dataframe_train), "AUC": ids.score_auc(quant_dataframe_train)},
    {"Data": "Testing", "Accuracy": ids.score(quant_dataframe_test), "AUC": ids.score_auc(quant_dataframe_test)}
])
metric_df

/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:91: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_classifier.py:58: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, value in row.iteritems():
/content/pyIDS/pyids/algorithms/ids_clas

,Data,Accuracy,AUC
0,Training,0.939177,0.953168
1,Testing,0.942857,0.957895
